In [1]:
import sys
sys.path.append("/home/bob/projects/Web3HackerNetwork/sandbox/python/bob/")
import commit_stat_engineering

In [3]:
project_root_path = '/home/bob/projects/Web3HackerNetwork'
dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_commit_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

for stat in all_commit_stats[slice(1)]:
    print(stat)

{'commit': 'fb528905fb6448e2b502f7de8fba877a2a995f6d', 'totalFiles': 2, 'binFiles': 0, 'textFiles': 2, 'binBytes': 0, 'textLines': 26, 'totalBytes': 780, 'pctBinBytes': 0, 'pctTextBytes': 100.0, 'jsFiles': 2, 'jsLines': 26, 'pctJsFiles': 100.0, 'pctJsLines': 100.0, 'pctJsBytes': 100.0, 'rustFiles': 0, 'rustLines': 0, 'pctRustFiles': 0, 'pctRustLines': 0, 'pctRustBytes': 0}
